# Import 部分

In [1]:
import scipy.io as sio
import h5py
import numpy as np
import keras
from keras import Input,layers
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D,Flatten,Conv2D, Dropout, MaxPooling2D, Input,Conv3D, MaxPooling3D, Dense, BatchNormalization, AveragePooling3D
from keras.optimizers import Adam,RMSprop,SGD
# from keras.layers.normalization import BatchNormalization
# from keras.layers.advanced_activations import LeakyReLU
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Activation, Permute, Dropout,Bidirectional,Reshape
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D,LSTM,Conv3D,AveragePooling3D
# from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
# from tensorflow.keras.layers import BatchNormalization
# from tensorflow.keras.layers import SpatialDropout2D
# from tensorflow.keras.regularizers import l1_l2
# from tensorflow.keras.layers import Input, Flatten
# from tensorflow.keras.constraints import max_norm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
# from keras import backend as K
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn import linear_model
from google.colab import drive
drive.mount('/content/gdrive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# load数据

In [2]:

Xfu10Sep=[]
Xpre10Sep=[]
Xpost10Sep=[]
Xcon10Sep=[]
for index in range (24):
    filepostname='/content/gdrive/My Drive/Master DATA/allpost_afterv610connectivity'+str(index+1)+'.npy'
    fileprename='/content/gdrive/My Drive/Master DATA/allpre_afterv610connectivity'+str(index+1)+'.npy'
    filefuname='/content/gdrive/My Drive/Master DATA/allfu_afterv610connectivity'+str(index+1)+'.npy'
    fileconname='/content/gdrive/My Drive/Master DATA/allcontrol_afterv610connectivity'+str(index+1)+'.npy'
    Xfu10Sep.append(np.load(filefuname).reshape([10,30,128,128,1]))
    Xpre10Sep.append(np.load(fileprename).reshape([10,30,128,128,1]))
    Xpost10Sep.append(np.load(filepostname).reshape([10,30,128,128,1]))
    Xcon10Sep.append(np.load(fileconname).reshape([10,30,128,128,1]))
Yfu10Connectivity4Sep=[]
Ypre10Connectivity4Sep=[]
Ypost10Connectivity4Sep=[]
Ycon10Connectivity4Sep=[]
for index in range (24):
    filepostname='/content/gdrive/My Drive/Master DATA/allpost_afterv6resp4area10connectivity'+str(index+1)+'.npy'
    fileprename='/content/gdrive/My Drive/Master DATA/allpre_afterv6resp4area10connectivity'+str(index+1)+'.npy'
    filefuname='/content/gdrive/My Drive/Master DATA/allfu_afterv6resp4area10connectivity'+str(index+1)+'.npy'
    fileconname='/content/gdrive/My Drive/Master DATA/allcontrol_afterv6resp4area10connectivity'+str(index+1)+'.npy'
    
    # filepostname='/content/gdrive/My Drive/Master DATA/allpost_afterv6resp9area'+str(index+1)+'.npy'
    # fileprename='/content/gdrive/My Drive/Master DATA/allpre_afterv6resp9area'+str(index+1)+'.npy'
    # filefuname='/content/gdrive/My Drive/Master DATA/allfu_afterv6resp9area'+str(index+1)+'.npy'
    # fileconname='/content/gdrive/My Drive/Master DATA/allcontrol_afterv6resp9area'+str(index+1)+'.npy'


    Yfu10Connectivity4Sep.append(np.load(filefuname))
    Ypre10Connectivity4Sep.append(np.load(fileprename))
    Ypost10Connectivity4Sep.append(np.load(filepostname))
    Ycon10Connectivity4Sep.append(np.load(fileconname))



In [ ]:
aa = np.array(Xcon10Sep)
bb = np.array(Ycon10Connectivity4Sep)
# print(Ycon10Connectivity4Sep[0])
print(len(Xcon10Sep))
print(aa.shape)
print(bb[0].shape)
# for i in range(24):


24
(24, 10, 30, 128, 128, 1)
(127, 9)


# 模型函数

In [3]:
def baseline_model_connect():
      # create model  
    x_input = Input(shape = (30,128,128,1))
    x = layers.Conv3D(3, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='lecun_uniform')(x_input) 
    x = layers.MaxPooling3D(pool_size=(2, 2, 2))(x)
    x = layers.BatchNormalization()(x)
    # x= layers.Dropout(0.6)(x)
    x = layers.Conv3D(3, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='lecun_uniform')(x)
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(x)
    x = layers.BatchNormalization()(x)
    # x= layers.Dropout(0.6)(x)
    x = layers.Conv3D(3, kernel_size=(4, 4, 4), activation='relu', kernel_initializer='lecun_uniform')(x)
    x = layers.AveragePooling3D(pool_size=(2, 2, 2))(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu', kernel_initializer='lecun_uniform')(x) 
    x = layers.Dense(512, activation='relu', kernel_initializer='lecun_uniform')(x)   
    y = layers.Dense(4, activation='sigmoid', name = 'y')(x)

    model=Model(inputs=x_input, outputs=y)
    model.summary() 
    model.compile(optimizer = Adam(lr=0.003),
                loss=['binary_crossentropy'],
                metrics=['accuracy'])
    return model

# 正磊原来的模型函数

In [ ]:
def baseline_model_connect():
      # create model  
    input1   = Input(shape = (30,128,128,1))

    

    ##################################################################


    block1       = Conv3D(8, kernel_size=(1,3,3), padding = 'same', 
                                   input_shape = (30,128,128,1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('relu')(block1)
    block1       = AveragePooling3D((1,2,2))(block1)
    block1       = Dropout(0.5)(block1)

    
    block2       = Conv3D(32, (1,3,3), use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling3D((1,2,2))(block2)
    block2       = Dropout(0.5)(block2)
    
    block3       = Conv3D(128, (1,3,3), use_bias = False, padding = 'same')(block2)
    block3       = BatchNormalization(axis = 1)(block3)
    block3       = Activation('elu')(block3)
    block3       = AveragePooling3D((1,2,2))(block3)
    block3       = Dropout(0.5)(block3)

    block4       = Conv3D(32, (1,3,3), use_bias = False, padding = 'same')(block3)
    block4       = BatchNormalization(axis = 1)(block4)
    block4       = Activation('elu')(block4)
    block4       = AveragePooling3D((1,2,2))(block4)
    block4       = Dropout(0.5)(block4)

    block5       = Conv3D(8, (1,3,3), use_bias = False, padding = 'same')(block4)
    block5       = BatchNormalization(axis = 1)(block5)
    block5       = Activation('elu')(block5)
    block5       = AveragePooling3D((1,2,2))(block5)
    block5       = Dropout(0.5)(block5)

    # block6       = Conv3D(2, (1,3,3), use_bias = False, padding = 'same')(block5)
    # block6       = BatchNormalization(axis = 1)(block6)
    # block6       = Activation('elu')(block6)
    # block6       = AveragePooling3D((1,2,2))(block6)
    # # block5       = Dropout(0.2)(block5)
    flatten      = Flatten(name = 'flatten')(block5)
    
    # dense        = Dense(128, kernel_initializer='normal',activation='relu')(flatten)
    dense        = Reshape((1,3840))(flatten)
    dense        = Bidirectional(LSTM(128, return_sequences=True))(dense)
    dense        = Bidirectional(LSTM(128))(dense)
    # dense        = Dense(32, kernel_initializer='normal',activation='relu')(flatten)
    # dense        = Reshape((1,3840))(flatten)
    # dense        = Bidirectional(LSTM(10, return_sequences=True))(dense)
    # dense        = Bidirectional(LSTM(10))(dense)
    dense        = Dense(16, kernel_initializer='normal',activation='relu')(dense)
    # dense        = Reshape((1,16))(dense)
    # dense        = Bidirectional(LSTM(10, return_sequences=True))(dense)
    # dense        = Bidirectional(LSTM(10))(dense)
    dense        = Dense(4, kernel_initializer='normal',activation='sigmoid')(dense)

    model=Model(inputs=input1, outputs=dense)
    model.summary()
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
    #model.compile(loss='mean_squared_error', optimizer='adam')
    return model

# 运行

In [5]:
shufflenumber=np.arange(24)
print(shufflenumber)
np.random.shuffle(shufflenumber)
print(shufflenumber)
k=4
num_val_samples=24//k
group=[]
score=[]

for index in range(k):
  temp=shufflenumber[index*num_val_samples:(index+1) * num_val_samples]
  group.append(temp)
print(group[3])
Xgroup=[]
Ygroup=[]
test_data=[]
test_label=[]
train_data=[] 
train_label=[]
for index in range(k):
  Xtemp=[]
  Ytemp=[]
  for index2 in group[index]:
    Xtemp.append(np.concatenate((Xfu10Sep[index2],Xpre10Sep[index2],Xpost10Sep[index2]),0))
    Ytemp.append(np.concatenate((Yfu10Connectivity4Sep[index2],Ypre10Connectivity4Sep[index2],Ypost10Connectivity4Sep[index2]),0))
  Xgroup.append(np.concatenate((Xtemp[0],Xtemp[1],Xtemp[2],Xtemp[3],Xtemp[4],Xtemp[5]),0))
  Ygroup.append(np.concatenate((Ytemp[0],Ytemp[1],Ytemp[2],Ytemp[3],Ytemp[4],Ytemp[5]),0))
print(Xgroup[0].shape)
print(Ygroup[0].shape)

for index in range(4):
  print('processing fold #', index)
  test_data = Xgroup[index]
  test_label = Ygroup[index]
  if index==0:
    train_data = np.concatenate((Xgroup[1],Xgroup[2],Xgroup[3]), axis=0)
    train_label = np.concatenate((Ygroup[1],Ygroup[2],Ygroup[3]), axis=0)
  elif index==1:
    train_data = np.concatenate((Xgroup[0],Xgroup[2],Xgroup[3]), axis=0)
    train_label = np.concatenate((Ygroup[0],Ygroup[2],Ygroup[3]), axis=0)
  elif index==2:
    train_data = np.concatenate((Xgroup[0],Xgroup[1],Xgroup[3]), axis=0)
    train_label = np.concatenate((Ygroup[0],Ygroup[1],Ygroup[3]), axis=0)
  else:
    train_data = np.concatenate((Xgroup[0],Xgroup[1],Xgroup[2]), axis=0)
    train_label = np.concatenate((Ygroup[0],Ygroup[1],Ygroup[2]), axis=0)

  model = baseline_model_connect()
  print(train_data.shape)
  print(train_label.shape)
  train_data,train_label = shuffle(train_data,train_label, random_state=2)
  #train_data=train_data.reshape([len(train_data),128,128,1])
  history = model.fit(train_data,train_label, epochs=50, batch_size=10,  verbose=1, validation_split=0.2)
  prd_r=model.predict(test_data)
  num_label = 4
  prd_r1=np.zeros(shape=[len(prd_r),num_label])
  
  for index in range (len(prd_r)):
    for index2 in range (num_label):
      if prd_r[index][index2]<0.5:
        prd_r1[index][index2]=0
      else:
        prd_r1[index][index2]=1
  score.append(accuracy_score(test_label.reshape([len(test_label)*num_label]),prd_r1.reshape([len(test_label)*num_label])))
  print(score)

print(score)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 3 23  5  1 15  2  0 11 21  9 22  8 19  6 20 13 17 18 10  7 16 12 14  4]
[10  7 16 12 14  4]
(180, 30, 128, 128, 1)
(180, 4)
processing fold # 0
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 30, 128, 128, 1)   0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 27, 125, 125, 3)   195       
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 13, 62, 62, 3)     0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 13, 62, 62, 3)     12        
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 10, 59, 59, 3)     579       
_____________________________________